In [1]:
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 33.0 MB/s eta 0:00:00


In [1]:
import flwr as fl
from flwr.common import Metrics
from flwr.common.typing import NDArrays, Scalar
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import librosa

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/daic/ProjectPrototype

/content/gdrive/MyDrive/daic/ProjectPrototype


In [4]:
!ls

'212148conf copy 3.ipynb'   edaicwoz   MFCCs_1030   MFCCs_1030.zip   preprocess_data.ipynb


In [5]:
train_labels_df = pd.read_csv("edaicwoz/train_split.csv")
test_labels_df = pd.read_csv("edaicwoz/test_split.csv")
val_labels_df = pd.read_csv("edaicwoz/dev_split.csv")

In [6]:
def load_audio_files(data_dir, sr=16000):
    file_ids = os.listdir(data_dir)
    subject_ids = []
    file_paths = []
    types = []
    labels = []
    labels_binary = []

    for file_id in file_ids:
        file_id = file_id.split("_")[0]
        file_path = [data_dir + "/" + file_id + "/" + file_id + "_MFCC_" + str(i) + ".npy" for i in range(len(next(iter(enumerate(os.walk(data_dir + "/" + str(file_id) + "/"))))[1][2]))]
        if int(file_id) in train_labels_df["Participant_ID"].values:
            types.append(0)
            labels.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])

        elif int(file_id) in test_labels_df["Participant_ID"].values:
            types.append(1)
            labels.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        else:
            types.append(2)
            labels.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        subject_ids.append(int(file_id))
        file_paths.append(file_path)

    return file_ids, subject_ids, file_paths, types, labels, labels_binary

data_dir = "MFCCs_1030"

file_ids, subject_ids, file_paths, types, labels, labels_binary = load_audio_files(data_dir)

In [7]:
def prepare_audio_set(file_paths):

    samples = []
    samples_ids = []
    samples_types = []
    samples_labels = []
    samples_labels_binary = []

    for i, file_path in enumerate(file_paths):
        all_mfccs = []
        for j in range(len(file_path)):
            all_mfccs.append(np.load(file_path[j]))
        all_mfccs = np.array(all_mfccs)
        samples.extend(all_mfccs)
        samples_ids.extend([subject_ids[i]] * len(all_mfccs))
        samples_types.extend([types[i]] * len(all_mfccs))
        samples_labels.extend([labels[i]] * len(all_mfccs))
        samples_labels_binary.extend([labels_binary[i]] * len(all_mfccs))

    samples = np.array(samples)

    samples_ids = np.array(samples_ids)
    samples_types = np.array(samples_types)
    samples_labels = np.array(samples_labels)
    samples_labels_binary = np.array(samples_labels_binary)

    return samples, samples_ids, samples_types, samples_labels, samples_labels_binary

print("[INFO] preparing data...")
samples, samples_ids, samples_types, samples_labels, samples_labels_binary = prepare_audio_set(file_paths)
samples = np.swapaxes(samples, 1, 2)

[INFO] preparing data...


In [8]:
training_samples = samples[samples_types == 0]
training_labels = samples_labels_binary[samples_types == 0]
training_subject_ids = samples_ids[samples_types == 0]

test_samples = samples[samples_types == 1]
test_labels = samples_labels_binary[samples_types == 1]

val_samples = samples[samples_types == 2]
val_labels_df = samples_labels_binary[samples_types == 2]

In [9]:
del samples, samples_ids, samples_types, samples_labels, samples_labels_binary
del file_ids, subject_ids, file_paths, types, labels, labels_binary

In [10]:
SEED = 42
NUM_CLIENTS = 8
BATCH_SIZE = 32
NUM_ROUNDS = 5
DEPRESSIVE_MULTIPLIER = 30
NON_DEPRESSIVE_MULTIPLIER = 10

In [11]:
def partition_data(X: np.ndarray, X_ids: np.ndarray, n_clients: int, d_mult: int, n_d_mult: int) -> Tuple[List[np.ndarray], List[np.ndarray]]:

    unique_subject_ids, counts = np.unique(X_ids, return_counts=True)

    # grouping the training samples by patient
    training_samples_grouped = []
    for i in unique_subject_ids:
        training_samples_grouped.append(X[X_ids == i])
    training_samples_grouped = np.array(training_samples_grouped)

    mask_30_segments = np.array([sample.shape[0] == DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    # creating masks to get deppressives and non deppressives
    mask_10_segments = np.array([sample.shape[0] == NON_DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    data_array_30_segments = training_samples_grouped[mask_30_segments]

    data_array_10_segments = training_samples_grouped[mask_10_segments]

    # recreating labels
    X_train_zeros = np.array_split(data_array_10_segments, NUM_CLIENTS)
    X_train_ones = np.array_split(data_array_30_segments, NUM_CLIENTS)

    # concatenating splitted ones and zeros with labels
    X_train_splitted = [] # (NUM_CLIENTS, data)
    y_train_splitted = [] # (NUM_CLIENTS, labels)
    for i in range(NUM_CLIENTS):

        # stack the segments from groups then concatenate
        client_data = np.concatenate((np.vstack(X_train_zeros[i]), np.vstack(X_train_ones[i])), axis=0)
        client_labels = np.concatenate((np.zeros((X_train_zeros[i].shape[0] * NON_DEPRESSIVE_MULTIPLIER), dtype=int), np.ones((X_train_ones[i].shape[0] * DEPRESSIVE_MULTIPLIER), dtype=int)), axis=0)

        X_train_splitted.append(client_data)
        y_train_splitted.append(client_labels)

    return X_train_splitted, y_train_splitted

X_trains, y_trains = partition_data(training_samples, training_subject_ids, NUM_CLIENTS, DEPRESSIVE_MULTIPLIER, NON_DEPRESSIVE_MULTIPLIER)

<ipython-input-11-45e26c792d34>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_samples_grouped = np.array(training_samples_grouped)


In [12]:
del training_samples, training_labels, training_subject_ids

In [ ]:
X_trains[0].shape

(610, 15001, 13)

In [13]:
def get_model(input_shape):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.GRU(units = 64, input_shape = input_shape))

    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    return model

In [14]:
from flwr.common.typing import NDArrays
class FlowerClient(fl.client.NumPyClient):

    def __init__(self, model: tf.keras.models.Sequential, X_train: np.ndarray, y_train: np.ndarray):
        self.model = model

        self.X_train = X_train
        self.y_train = y_train

    def get_parameters(self, config):
        return self.model.get_weights()


    def fit(self, parameters: NDArrays, config: Dict[str, Scalar]) -> NDArrays:

        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

        self.model.set_weights(parameters)

        history = self.model.fit(self.X_train, self.y_train ,batch_size=BATCH_SIZE, epochs=1, verbose=0)
        results = {
            "loss": history.history["loss"][0],
            "accuracy": history.history["accuracy"][0],
        }
        return self.model.get_weights(), len(self.X_train), results

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar])-> Tuple[float, int, Dict[str, Scalar]]:
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
        self.model.set_weights(parameters)

        loss, acc = self.model.evaluate(self.X_train, self.y_train, verbose=0)
        return loss, len(self.X_train), {"accuracy": acc}

In [15]:
# client creator by client id
def create_client_fn(cid: str) -> FlowerClient:

    input_shape = (15001, 13)
    model = get_model(input_shape)
    cid_int = int(cid)
    return FlowerClient(model, X_trains[cid_int], y_trains[cid_int])

In [16]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [17]:
patience = 5
best_accuracy = 0.0
best_loss = 999
weights = np.array([])

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    """Centralized evaluation function"""

    input_shape = (15001, 13)
    model = get_model(input_shape)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

    model.set_weights(parameters)

    loss, accuracy = model.evaluate(val_samples, val_labels_df, batch_size=16, verbose=0)

    global best_accuracy
    global best_loss
    global weights

    print(f"LOSS: {loss}")
    print(f"BEST_LOSS: {best_loss}")
    print(f"ACCURACY: {accuracy}")
    print(f"BEST_ACCURACY: {best_accuracy}")

    if loss < best_loss:
        best_accuracy = accuracy
        weights = parameters
        best_loss = loss

    return loss, {"accuracy": accuracy}


client_resources = {"num_cpus": 2}
if tf.config.get_visible_devices("GPU"):
    client_resources["num_gpus"] = 1

# Specify the Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1.0,  
    min_fit_clients=NUM_CLIENTS,  
    min_evaluate_clients=NUM_CLIENTS,  
    min_available_clients=NUM_CLIENTS,  # Wait until all 8 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
)

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=create_client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=15),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-12 15:18:10,718 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
2023-12-12 15:18:12,876	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-12 15:18:14,171 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 32540457371.0, 'node:__internal_head__': 1.0, 'CPU': 8.0, 'object_store_memory': 16270228684.0, 'accelerator_type:V100': 1.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 32540457371.0, 'node:__internal_head__': 1.0, 'CPU': 8.0, 'object_store_memory': 16270228684.0, 'accelerator_type:V100': 1.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-12-12 15:18:14,173 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower V

LOSS: 0.717132031917572
BEST_LOSS: 999
ACCURACY: 0.4675000011920929
BEST_ACCURACY: 0.0


DEBUG flwr 2023-12-12 15:20:27,887 | server.py:236 | fit_round 1 received 8 results and 0 failures
DEBUG:flwr:fit_round 1 received 8 results and 0 failures
WARNING flwr 2023-12-12 15:20:27,900 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-12-12 15:20:36,491 | server.py:125 | fit progress: (1, 0.6831395030021667, {'accuracy': 0.5487499833106995}, 117.71515408899995)
INFO:flwr:fit progress: (1, 0.6831395030021667, {'accuracy': 0.5487499833106995}, 117.71515408899995)
DEBUG flwr 2023-12-12 15:20:36,495 | server.py:173 | evaluate_round 1: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 1: strategy sampled 8 clients (out of 8)


LOSS: 0.6831395030021667
BEST_LOSS: 0.717132031917572
ACCURACY: 0.5487499833106995
BEST_ACCURACY: 0.4675000011920929


DEBUG flwr 2023-12-12 15:21:56,295 | server.py:187 | evaluate_round 1 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 8 results and 0 failures
DEBUG flwr 2023-12-12 15:21:56,298 | server.py:222 | fit_round 2: strategy sampled 8 clients (out of 8)
DEBUG:flwr:fit_round 2: strategy sampled 8 clients (out of 8)
DEBUG flwr 2023-12-12 15:23:39,669 | server.py:236 | fit_round 2 received 8 results and 0 failures
DEBUG:flwr:fit_round 2 received 8 results and 0 failures
INFO flwr 2023-12-12 15:23:47,954 | server.py:125 | fit progress: (2, 0.6764363646507263, {'accuracy': 0.5724999904632568}, 309.17868630799967)
INFO:flwr:fit progress: (2, 0.6764363646507263, {'accuracy': 0.5724999904632568}, 309.17868630799967)
DEBUG flwr 2023-12-12 15:23:47,956 | server.py:173 | evaluate_round 2: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 2: strategy sampled 8 clients (out of 8)


...

DEBUG flwr 2023-12-12 16:00:14,174 | server.py:187 | evaluate_round 13 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 13 received 8 results and 0 failures
DEBUG flwr 2023-12-12 16:00:14,178 | server.py:222 | fit_round 14: strategy sampled 8 clients (out of 8)
DEBUG:flwr:fit_round 14: strategy sampled 8 clients (out of 8)
DEBUG flwr 2023-12-12 16:02:01,354 | server.py:236 | fit_round 14 received 8 results and 0 failures
DEBUG:flwr:fit_round 14 received 8 results and 0 failures
INFO flwr 2023-12-12 16:02:09,917 | server.py:125 | fit progress: (14, 0.671558678150177, {'accuracy': 0.5887500047683716}, 2611.1417991399994)
INFO:flwr:fit progress: (14, 0.671558678150177, {'accuracy': 0.5887500047683716}, 2611.1417991399994)
DEBUG flwr 2023-12-12 16:02:09,920 | server.py:173 | evaluate_round 14: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 14: strategy sampled 8 clients (out of 8)


LOSS: 0.671558678150177
BEST_LOSS: 0.670042097568512
ACCURACY: 0.5887500047683716
BEST_ACCURACY: 0.6050000190734863


DEBUG flwr 2023-12-12 16:03:30,033 | server.py:187 | evaluate_round 14 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 14 received 8 results and 0 failures
DEBUG flwr 2023-12-12 16:03:30,035 | server.py:222 | fit_round 15: strategy sampled 8 clients (out of 8)
DEBUG:flwr:fit_round 15: strategy sampled 8 clients (out of 8)
DEBUG flwr 2023-12-12 16:05:15,388 | server.py:236 | fit_round 15 received 8 results and 0 failures
DEBUG:flwr:fit_round 15 received 8 results and 0 failures
INFO flwr 2023-12-12 16:05:23,817 | server.py:125 | fit progress: (15, 0.6711401343345642, {'accuracy': 0.5962499976158142}, 2805.041807269)
INFO:flwr:fit progress: (15, 0.6711401343345642, {'accuracy': 0.5962499976158142}, 2805.041807269)
DEBUG flwr 2023-12-12 16:05:23,820 | server.py:173 | evaluate_round 15: strategy sampled 8 clients (out of 8)
DEBUG:flwr:evaluate_round 15: strategy sampled 8 clients (out of 8)


LOSS: 0.6711401343345642
BEST_LOSS: 0.670042097568512
ACCURACY: 0.5962499976158142
BEST_ACCURACY: 0.6050000190734863


DEBUG flwr 2023-12-12 16:06:43,651 | server.py:187 | evaluate_round 15 received 8 results and 0 failures
DEBUG:flwr:evaluate_round 15 received 8 results and 0 failures
INFO flwr 2023-12-12 16:06:43,653 | server.py:153 | FL finished in 2884.8771971939996
INFO:flwr:FL finished in 2884.8771971939996
INFO flwr 2023-12-12 16:06:43,660 | app.py:226 | app_fit: losses_distributed [(1, 0.6851071611363837), (2, 0.6804177451641001), (3, 0.6760555805043972), (4, 0.673114810091384), (5, 0.6705778114339138), (6, 0.6675135741842554), (7, 0.6652790604753697), (8, 0.6628408269679292), (9, 0.6610716545835454), (10, 0.6587436549206998), (11, 0.6563230364880663), (12, 0.6541205147479443), (13, 0.6512731384723744), (14, 0.6487849301480232), (15, 0.6466497852447185)]
INFO:flwr:app_fit: losses_distributed [(1, 0.6851071611363837), (2, 0.6804177451641001), (3, 0.6760555805043972), (4, 0.673114810091384), (5, 0.6705778114339138), (6, 0.6675135741842554), (7, 0.6652790604753697), (8, 0.6628408269679292), (9, 0.

In [18]:
history

History (loss, distributed):
	round 1: 0.6851071611363837
	round 2: 0.6804177451641001
	round 3: 0.6760555805043972
	round 4: 0.673114810091384
	round 5: 0.6705778114339138
	round 6: 0.6675135741842554
	round 7: 0.6652790604753697
	round 8: 0.6628408269679292
	round 9: 0.6610716545835454
	round 10: 0.6587436549206998
	round 11: 0.6563230364880663
	round 12: 0.6541205147479443
	round 13: 0.6512731384723744
	round 14: 0.6487849301480232
	round 15: 0.6466497852447185
History (loss, centralized):
	round 0: 0.717132031917572
	round 1: 0.6831395030021667
	round 2: 0.6764363646507263
	round 3: 0.6748592853546143
	round 4: 0.6728706359863281
	round 5: 0.6714292764663696
	round 6: 0.6725345849990845
	round 7: 0.6725710034370422
	round 8: 0.6716809272766113
	round 9: 0.6707581877708435
	round 10: 0.6713998913764954
	round 11: 0.670581042766571
	round 12: 0.670042097568512
	round 13: 0.6702312231063843
	round 14: 0.671558678150177
	round 15: 0.6711401343345642
History (metrics, distributed, evalu

In [19]:
# printing validation results
print(best_accuracy)
print(best_loss)
print(patience)
print(weights)

0.6050000190734863
0.670042097568512
5
[array([[-0.11188927,  0.04626382, -0.10107277, ...,  0.02708078,
        -0.12880059,  0.12052274],
       [ 0.10582084,  0.14268203, -0.11345811, ..., -0.11299951,
        -0.01334307,  0.05342008],
       [-0.12120803, -0.05436783, -0.14651234, ...,  0.0409455 ,
        -0.08060317, -0.14220315],
       ...,
       [ 0.04401699, -0.1459625 , -0.15326001, ...,  0.11114775,
         0.06289359,  0.05430939],
       [-0.11394229,  0.0289547 , -0.142676  , ..., -0.04175366,
         0.124612  , -0.16681795],
       [-0.1231417 , -0.13935745, -0.10235354, ...,  0.0997204 ,
        -0.04045389, -0.09139772]], dtype=float32), array([[ 0.13017482, -0.07719965, -0.09669306, ..., -0.01106421,
        -0.10978445,  0.07405293],
       [-0.08135707, -0.06577331, -0.02556281, ..., -0.02791366,
        -0.05424706,  0.11066396],
       [ 0.01802721,  0.01223328,  0.12031075, ..., -0.00230684,
        -0.05532712,  0.10695595],
       ...,
       [-0.02828089